In [1]:
import pandas as pd
from statistics import median
import random
import csv
from projects import project_list
from collections import Counter
from batch_updation import *
from batching_algs import *

In [2]:
previous_state = {
                'BATCHBISECT': {'linear': {1:0, 2:0, 3:0, 4:0}, 'exponential':{2:0, 3:0, 4:0}, 'stagger':{2:0, 3:0}, 
               'random_linear':{-1:0}, 'random_exponential':{-1:0}, 'random_random':{-1:0}}, 
               'BATCHSTOP4': {'linear': {1:0, 2:0, 3:0, 4:0}, 'exponential':{2:0, 3:0, 4:0}, 'stagger':{2:0, 3:0}, 
               'random_linear':{-1:0}, 'random_exponential':{-1:0}, 'random_random':{-1:0}},
                'BATCHDIVIDE4' : {'linear': {1:0, 2:0, 3:0, 4:0}, 'exponential':{2:0, 3:0, 4:0}, 'stagger':{2:0, 3:0}, 
               'random_linear':{-1:0}, 'random_exponential':{-1:0}, 'random_random':{-1:0}} }

In [3]:
batch_total = 0
batch_duration = 0

In [4]:
def output_values(tr_results):
    l = []
    i = 0
    while i < len(tr_results):
        if tr_results[i] == 'passed':
            l.append(1)
        else:
            l.append(0)
        i += 1
    
    return l

In [5]:
def dynamic_batch(commits, print_flag =0, reset_flag=0):
    
    global linear_update, exp_update, random_linear, random_exp, freq_update, random_random
    global batchbisect, batchstop4, batchdivide4
    global batch_total
    
    
    rows = []
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    lines = 0
    
    algorithms = {'BATCHBISECT': batchbisect, 'BATCHSTOP4':batchstop4, 'BATCHDIVIDE4':batchdivide4}
    methods = {'linear': linear_update, 'exponential':exp_update, 'stagger':stagger_update, 
               'random_linear':random_linear, 'random_exponential':random_exp, 'random_random':random_random,
               }

    for m in methods:
        for alg in algorithms:
            
            
            
            if m == 'linear':
                factors = [1, 2, 3, 4]
                
                for f in factors:
                    if reset_flag == 1:
                        batch_size = previous_state[alg][m][f]
                    else:
                        batch_size = 16
            
                    builds = 0
                    delays = []
                    i = 0

                    while i < length:
                        batch = commits[i:i+batch_size]

                        delays.extend([batch_size-x-1 for x in range(batch_size)])

                        batch_total = algorithms[alg](batch)
                        builds += batch_total
#                         print(alg, m)
#                         print(batch_size, batch, batch_total, builds)

                        i += batch_size
                        
                        if 0 in batch:
                            res = 0
                        else:
                            res = 1
                        #updating batch size
                        batch_size = methods[m](res, batch, batch_size, f)

                    saved = 100-(100*builds/length)
                    fr = 100*results.count(0)/length
                    rows.append([p_name, str(alg), '-1', str(m), f, builds, saved, length])
                    lines += 1
                    
                    print('Updating batch size for {}{}{} to be {}'.format(alg, m, f, batch_size) )
                    previous_state[alg][m][f] = batch_size
            
            elif (m == 'exponential') or (m == 'stagger'):
                factors = [2, 3, 4]
                
                for f in factors:
                    
                    if reset_flag == 1:
                        batch_size = previous_state[alg][m][f]
                    else:
                        batch_size = 16
            
                    builds = 0
                    delays = []
                    i = 0

                    while i < length:
                        batch = commits[i:i+batch_size]

                        delays.extend([batch_size-x-1 for x in range(batch_size)])

                        batch_total = algorithms[alg](batch)
                        builds += batch_total
#                         print(alg, m)
#                         print(batch_size, batch, batch_total, builds)

                        i += batch_size
                        if 0 in batch:
                            res = 0
                        else:
                            res = 1

                        #updating batch size
                        batch_size = methods[m](res, batch, batch_size, f)

                    saved = 100-(100*builds/length)
                    fr = 100*results.count(0)/length
                    rows.append([p_name, str(alg), '-1', str(m), f, builds, saved, length])
                    lines += 1
                    
                    print('Updating batch size for {}{}{} to be {}'.format(alg, m, f, batch_size) )
                    previous_state[alg][m][f] = batch_size
            
            else:
                
                builds = 0
                delays = []
                i = 0
                
                if reset_flag == 1:
                    batch_size = previous_state[alg][m][-1]
                else:
                    batch_size = 16
                
                while i < length:
                    batch = commits[i:i+batch_size]

                    delays.extend([batch_size-x-1 for x in range(batch_size)])

                    batch_total = algorithms[alg](batch)
                    builds += batch_total
#                     print(alg, m)
#                     print(batch_size, batch, batch_total, builds)

                    i += batch_size
                    if 0 in batch:
                        res = 0
                    else:
                        res = 1

                    #updating batch size
                    batch_size = methods[m](res, batch, batch_size, -1)

                saved = 100-(100*builds/length)
                fr = 100*results.count(0)/length
                rows.append([p_name, str(alg), '-1', str(m), -1, builds, saved, length])
                lines += 1
                
                print('Updating batch size for {}{}{} to be {}'.format(alg, m, f, batch_size) )
                previous_state[alg][m][-1] = batch_size
            
            #print([p_name, alg, '-1', m, saved, length])
    #print(lines)
    return rows
            

In [6]:
i = 0

first_half_rows = []
second_half_rows = []
for p in project_list:
    p_name = p.split('/')[1]
    
    if (p_name == 'BuildCraft-BuildCraft.csv') | (p_name == 'RS485-LogisticsPipes.csv') | (p_name == 'datastax-java-driver.csv') :
        continue
        
    data = pd.read_csv('../dynamic batching/ci_skip_data/extracted_project_travis/' + p_name)
    
    branch_type = data['git_branch'].tolist()
    if 'master' in branch_type:
        data = data[ data['git_branch'] == 'master']
    else:
        data = data[ data['git_branch'] == 'trunk']
        
    total_length = len(data) - 100
    quarter_length = total_length//4
    
    run_data = data.iloc[100:quarter_length]
    if len(run_data) == 0:
        print('out of data')
        i += 1
        continue
    results = output_values(run_data['tr_status'].tolist())
    duration = run_data['tr_duration'].tolist()
    
    
    rows = dynamic_batch(results)
    first_half_rows.extend(rows)
    
    
    run_data = data.iloc[len(data)-quarter_length:]
    if len(run_data) == 0:
        i += 1
        continue
    results = output_values(run_data['tr_status'].tolist())
    rows = dynamic_batch(results, print_flag=1, reset_flag = 1)
    second_half_rows.extend(rows)
    
    
    
    

Updating batch size for BATCHBISECTlinear1 to be 15
Updating batch size for BATCHBISECTlinear2 to be 14
Updating batch size for BATCHBISECTlinear3 to be 13
Updating batch size for BATCHBISECTlinear4 to be 12
Updating batch size for BATCHSTOP4linear1 to be 15
Updating batch size for BATCHSTOP4linear2 to be 14
Updating batch size for BATCHSTOP4linear3 to be 13
Updating batch size for BATCHSTOP4linear4 to be 12
Updating batch size for BATCHDIVIDE4linear1 to be 15
Updating batch size for BATCHDIVIDE4linear2 to be 14
Updating batch size for BATCHDIVIDE4linear3 to be 13
Updating batch size for BATCHDIVIDE4linear4 to be 12
Updating batch size for BATCHBISECTexponential2 to be 16
Updating batch size for BATCHBISECTexponential3 to be 5
Updating batch size for BATCHBISECTexponential4 to be 16
Updating batch size for BATCHSTOP4exponential2 to be 16
Updating batch size for BATCHSTOP4exponential3 to be 5
Updating batch size for BATCHSTOP4exponential4 to be 16
Updating batch size for BATCHDIVIDE4exp

Updating batch size for BATCHDIVIDE4exponential3 to be 1
Updating batch size for BATCHDIVIDE4exponential4 to be 1
Updating batch size for BATCHBISECTstagger2 to be 1
Updating batch size for BATCHBISECTstagger3 to be 1
Updating batch size for BATCHBISECTstagger4 to be 1
Updating batch size for BATCHSTOP4stagger2 to be 1
Updating batch size for BATCHSTOP4stagger3 to be 1
Updating batch size for BATCHSTOP4stagger4 to be 1
Updating batch size for BATCHDIVIDE4stagger2 to be 1
Updating batch size for BATCHDIVIDE4stagger3 to be 1
Updating batch size for BATCHDIVIDE4stagger4 to be 1
Updating batch size for BATCHBISECTrandom_linear4 to be 1
Updating batch size for BATCHSTOP4random_linear4 to be 1
Updating batch size for BATCHDIVIDE4random_linear4 to be 1
Updating batch size for BATCHBISECTrandom_exponential4 to be 1
Updating batch size for BATCHSTOP4random_exponential4 to be 1
Updating batch size for BATCHDIVIDE4random_exponential4 to be 1
Updating batch size for BATCHBISECTrandom_random4 to be

Updating batch size for BATCHBISECTstagger2 to be 1
Updating batch size for BATCHBISECTstagger3 to be 1
Updating batch size for BATCHBISECTstagger4 to be 1
Updating batch size for BATCHSTOP4stagger2 to be 1
Updating batch size for BATCHSTOP4stagger3 to be 1
Updating batch size for BATCHSTOP4stagger4 to be 1
Updating batch size for BATCHDIVIDE4stagger2 to be 1
Updating batch size for BATCHDIVIDE4stagger3 to be 1
Updating batch size for BATCHDIVIDE4stagger4 to be 1
Updating batch size for BATCHBISECTrandom_linear4 to be 19
Updating batch size for BATCHSTOP4random_linear4 to be 1
Updating batch size for BATCHDIVIDE4random_linear4 to be 1
Updating batch size for BATCHBISECTrandom_exponential4 to be 1
Updating batch size for BATCHSTOP4random_exponential4 to be 1
Updating batch size for BATCHDIVIDE4random_exponential4 to be 2
Updating batch size for BATCHBISECTrandom_random4 to be 7
Updating batch size for BATCHSTOP4random_random4 to be 1
Updating batch size for BATCHDIVIDE4random_random4 to

Updating batch size for BATCHSTOP4linear1 to be 3
Updating batch size for BATCHSTOP4linear2 to be 5
Updating batch size for BATCHSTOP4linear3 to be 7
Updating batch size for BATCHSTOP4linear4 to be 1
Updating batch size for BATCHDIVIDE4linear1 to be 3
Updating batch size for BATCHDIVIDE4linear2 to be 5
Updating batch size for BATCHDIVIDE4linear3 to be 7
Updating batch size for BATCHDIVIDE4linear4 to be 1
Updating batch size for BATCHBISECTexponential2 to be 4
Updating batch size for BATCHBISECTexponential3 to be 3
Updating batch size for BATCHBISECTexponential4 to be 4
Updating batch size for BATCHSTOP4exponential2 to be 4
Updating batch size for BATCHSTOP4exponential3 to be 3
Updating batch size for BATCHSTOP4exponential4 to be 4
Updating batch size for BATCHDIVIDE4exponential2 to be 4
Updating batch size for BATCHDIVIDE4exponential3 to be 3
Updating batch size for BATCHDIVIDE4exponential4 to be 4
Updating batch size for BATCHBISECTstagger2 to be 4
Updating batch size for BATCHBISECTs

Updating batch size for BATCHBISECTrandom_linear4 to be 24
Updating batch size for BATCHSTOP4random_linear4 to be 25
Updating batch size for BATCHDIVIDE4random_linear4 to be 16
Updating batch size for BATCHBISECTrandom_exponential4 to be 8
Updating batch size for BATCHSTOP4random_exponential4 to be 1
Updating batch size for BATCHDIVIDE4random_exponential4 to be 16
Updating batch size for BATCHBISECTrandom_random4 to be 15
Updating batch size for BATCHSTOP4random_random4 to be 16
Updating batch size for BATCHDIVIDE4random_random4 to be 14
Updating batch size for BATCHBISECTlinear1 to be 1
Updating batch size for BATCHBISECTlinear2 to be 1
Updating batch size for BATCHBISECTlinear3 to be 1
Updating batch size for BATCHBISECTlinear4 to be 1
Updating batch size for BATCHSTOP4linear1 to be 1
Updating batch size for BATCHSTOP4linear2 to be 1
Updating batch size for BATCHSTOP4linear3 to be 1
Updating batch size for BATCHSTOP4linear4 to be 1
Updating batch size for BATCHDIVIDE4linear1 to be 1


...

Updating batch size for BATCHBISECTlinear1 to be 1
Updating batch size for BATCHBISECTlinear2 to be 1
Updating batch size for BATCHBISECTlinear3 to be 1
Updating batch size for BATCHBISECTlinear4 to be 1
Updating batch size for BATCHSTOP4linear1 to be 1
Updating batch size for BATCHSTOP4linear2 to be 1
Updating batch size for BATCHSTOP4linear3 to be 1
Updating batch size for BATCHSTOP4linear4 to be 1
Updating batch size for BATCHDIVIDE4linear1 to be 1
Updating batch size for BATCHDIVIDE4linear2 to be 1
Updating batch size for BATCHDIVIDE4linear3 to be 1
Updating batch size for BATCHDIVIDE4linear4 to be 1
Updating batch size for BATCHBISECTexponential2 to be 1
Updating batch size for BATCHBISECTexponential3 to be 1
Updating batch size for BATCHBISECTexponential4 to be 1
Updating batch size for BATCHSTOP4exponential2 to be 1
Updating batch size for BATCHSTOP4exponential3 to be 1
Updating batch size for BATCHSTOP4exponential4 to be 1
Updating batch size for BATCHDIVIDE4exponential2 to be 

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,8,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



Updating batch size for BATCHSTOP4linear3 to be 1
Updating batch size for BATCHSTOP4linear4 to be 1
Updating batch size for BATCHDIVIDE4linear1 to be 1
Updating batch size for BATCHDIVIDE4linear2 to be 1
Updating batch size for BATCHDIVIDE4linear3 to be 1
Updating batch size for BATCHDIVIDE4linear4 to be 1
Updating batch size for BATCHBISECTexponential2 to be 1
Updating batch size for BATCHBISECTexponential3 to be 1
Updating batch size for BATCHBISECTexponential4 to be 1
Updating batch size for BATCHSTOP4exponential2 to be 1
Updating batch size for BATCHSTOP4exponential3 to be 1
Updating batch size for BATCHSTOP4exponential4 to be 1
Updating batch size for BATCHDIVIDE4exponential2 to be 1
Updating batch size for BATCHDIVIDE4exponential3 to be 1
Updating batch size for BATCHDIVIDE4exponential4 to be 1
Updating batch size for BATCHBISECTstagger2 to be 1
Updating batch size for BATCHBISECTstagger3 to be 1
Updating batch size for BATCHBISECTstagger4 to be 1
Updating batch size for BATCHST

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,8,20,51,53,61) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Updating batch size for BATCHBISECTlinear1 to be 16
Updating batch size for BATCHBISECTlinear2 to be 15
Updating batch size for BATCHBISECTlinear3 to be 16
Updating batch size for BATCHBISECTlinear4 to be 13
Updating batch size for BATCHSTOP4linear1 to be 16
Updating batch size for BATCHSTOP4linear2 to be 15
Updating batch size for BATCHSTOP4linear3 to be 16
Updating batch size for BATCHSTOP4linear4 to be 13
Updating batch size for BATCHDIVIDE4linear1 to be 16
Updating batch size for BATCHDIVIDE4linear2 to be 15
Updating batch size for BATCHDIVIDE4linear3 to be 16
Updating batch size for BATCHDIVIDE4linear4 to be 13
Updating batch size for BATCHBISECTexponential2 to be 16
Updating batch size for BATCHBISECTexponential3 to be 9
Updating batch size for BATCHBISECTexponential4 to be 16
Updating batch size for BATCHSTOP4exponential2 to be 16
Updating batch size for BATCHSTOP4exponential3 to be 9
Updating batch size for BATCHSTOP4exponential4 to be 16
Updating batch size for BATCHDIVIDE4exp

In [7]:
df = pd.DataFrame(first_half_rows, columns=['project', 'algorithm', 'batch_size', 'update_method', 'factor', 'num_of_builds', 'builds_saved', 'testall_size'])
df.to_csv('first_quarter_sd_noreset.csv')

In [8]:
df = pd.DataFrame(second_half_rows, columns=['project', 'algorithm', 'batch_size', 'update_method', 'factor', 'num_of_builds', 'builds_saved', 'testall_size'])
df.to_csv('second_quarter_sd_noreset.csv')

In [9]:
# def batchstop4_dynamic(commits):
#     global batch_total

#     batch_size = 16
#     length = len(commits)
#     builds = 0
#     i = 0
#     delays = []

#     while i < length:
#         batch = commits[i:i+batch_size]
#         fails_per = 100*batch.count(0)/batch_size
#         delays.extend([batch_size-x-1 for x in range(batch_size)])

#         batch_total = 0
#         res = batchstop4(batch)
#         builds += batch_total
#         print('Building {} {} {} {}'.format(batch_size, batch, batch_total, builds))

#         i += batch_size

#         #updating batch size
#         batch_size = linear_update(res, batch)

#     return builds, delays

In [10]:
# for p in project_list:
#     p_name = p.split('/')[1]
#     try:
#         data = pd.read_csv('ci_skip_data/extracted_project_travis/' + p_name)
#     except:
#         continue
#     total_length = len(data) - 100
#     data = data[ data['verdict'] == 0]
#     data = data.iloc[100:]
#     results = output_values(data['tr_status'].tolist())
#     builds, delays = batchstop4_dynamic(results)
#     reqd = 100-(100*builds/total_length)
#     fr = 100*results.count(0)/total_length
    
#     writer.writerow([p_name, 'BATCHSTOP4', '-1', reqd, median(delays), total_length])
    
#     #print('{:>40} {:>10} {:>30} {:>10}'.format(p, len(results), reqd, median(delays)))

In [11]:
# for p in project_list:
#     p_name = p.split('/')[1]
#     try:
#         data = pd.read_csv('ci_skip_data/extracted_project_travis/' + p_name)
#     except:
#         continue
#     total_length = len(data) - 100
#     #data = data[ data['verdict'] == 0]
#     data = data.iloc[100:]
#     results = output_values(data['tr_status'].tolist())
#     builds, delays = dynamic_batch(results)
#     reqd = 100-(100*builds/total_length)
#     fr = 100*results.count(0)/total_length
    
#     writer.writerow([p_name, 'BATCHBISECT', '-1', reqd, median(delays), total_length])
    
    